### Importing Libraries

In [ ]:
%load_ext sparksql_magic

import requests
import json
import pandas as pd
import time, datetime

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

In [30]:
class Collector:
    def __init__(self, start_date, end_date, coin):
        self.start_date = start_date
        self.end_date = end_date
        self.coin = coin
        self.spark = SparkSession.builder.appName("SparkCollectorPtax").getOrCreate()
        self.landing_zone_data = None

    def a(self):
        try:
            api_url = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@moeda='{self.coin}'&@dataInicial='{self.start_date}'&@dataFinalCotacao='{self.end_date}'&$top=10000&$filter=tipoBoletim%20eq%20'Fechamento'&$format=json&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim"
            response = requests.get(api_url)
            response.raise_for_status()
            
            if response.status_code == 200:
                data = response.json()
                folder_filename = "C:/Tecnology/Projects/bacen-ptax-data-processing/data/raw/ptax_data"
                with open(folder_filename, "w") as file:
                    json.dump(data, file)
                    self.landing_zone_data = True
            else:
                print('Failed to get data from API.')
                self.landing_zone_data = False
                
        except requests.exceptions.RequestException as e:
            print('Failed to get data from API. Error: ', e)
            self.landing_zone_data = False

    def a(self):
        folder_filename = "C:/Tecnology/Projects/bacen-ptax-data-processing/data/raw/ptax_data"
        folder_bronze = "C:/Tecnology/Projects/bacen-ptax-data-processing/data/processed/bronze/bronze.csv"

        df = self.spark.read \
            .json(folder_filename)
        
        df.createOrReplaceTempView("ptax")
        query = '''
            SELECT 
                val.cotacaoCompra, 
                val.cotacaoVenda, 
                val.DataHoraCotacao, 
                val.TipoBoletim 
            FROM ptax P
            LATERAL VIEW EXPLODE(P.value) AS val
        '''
        
        df_bronze = self.spark.sql(query)
        df_bronze.save("data", "csv", "overwrite")

    
        
    